# Dual-Network Hydrogel Mastercurve

Auto TTS on dual-network hydrogels across temperature.

**Data:** examples/data/pyRheo/dual_network_hydrogel/*.csv


In [1]:
# Google Colab compatibility - uncomment if running in Colab
# !pip install -q rheojax
# from google.colab import drive
# drive.mount('/content/drive')


## Setup and Imports

In [2]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
%matplotlib inline

import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from rheojax.core.data import RheoData
from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fractional_maxwell_model import FractionalMaxwellModel
from rheojax.models.generalized_maxwell import GeneralizedMaxwell
from rheojax.models.herschel_bulkley import HerschelBulkley
from rheojax.pipeline.base import Pipeline
from rheojax.transforms.mastercurve import Mastercurve

jax, jnp = safe_import_jax()
verify_float64()
np.set_printoptions(precision=4, suppress=True)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
warnings.filterwarnings('ignore', category=RuntimeWarning)

def r2_complex(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    ss_res = np.sum(np.abs(y_true - y_pred) ** 2)
    ss_tot = np.sum(np.abs(y_true - np.mean(y_true)) ** 2)
    return float(1 - ss_res / ss_tot)

def mpe(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return float(np.mean(np.abs(y_true - y_pred) / np.maximum(np.abs(y_true), 1e-12)) * 100)


INFO:2025-12-06 05:01:38,282:jax._src.xla_bridge:808: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Loading rheojax version 0.4.0


/Users/b80985/Projects/rheojax/.venv/lib/python3.13/site-packages/piblin_jax/__init__.py:28: UserWarning: GPU acceleration is only available on Linux with CUDA 12+. Using JAX in CPU mode.
  from . import backend, data, dataio, transform


## Load multi-temperature sweeps (UTF-16)

In [3]:
DATA_DIR = Path.cwd().parent / 'data' / 'pyRheo' / 'dual_network_hydrogel'
files = sorted(DATA_DIR.glob('*.csv'))

datasets = []
for fpath in files:
    stem = fpath.stem
    temp_token = stem.split('_')[-1]
    temp_c = float(temp_token.replace('c','').replace('h',''))
    df = pd.read_csv(fpath, encoding='utf-16', sep='	', skiprows=6, names=['omega', 'Gp', 'Gpp'], decimal=',')
    df = df.dropna()
    df['omega'] = pd.to_numeric(df['omega'], errors='coerce')
    df['Gp'] = pd.to_numeric(df['Gp'], errors='coerce')
    df['Gpp'] = pd.to_numeric(df['Gpp'], errors='coerce')
    df = df.dropna()
    if df.empty:
        continue
    omega = df['omega'].to_numpy()
    Gp = df['Gp'].to_numpy()
    Gpp = df['Gpp'].to_numpy()
    datasets.append(RheoData(x=omega, y=Gp + 1j*Gpp, x_units='rad/s', y_units='Pa', domain='oscillation', metadata={'temperature': temp_c + 273.15, 'test_mode': 'oscillation'}))

mc = Mastercurve(reference_temp=298.15, method='wlf', auto_shift=True)
try:
    master, shifts = mc.create_mastercurve(datasets, return_shifts=True)
except Exception as exc:
    print(f"Auto-shift failed: {exc}; falling back to manual identity shifts")
    manual_shifts = {d.metadata['temperature']: 1.0 for d in datasets}
    mc = Mastercurve(reference_temp=298.15, method='manual', auto_shift=False)
    mc.set_manual_shifts(manual_shifts)
    master, shifts = mc.create_mastercurve(datasets, return_shifts=True)

print(f"Temps (C): {[round(d.metadata['temperature']-273.15,1) for d in datasets]}")
print(shifts)


Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-06}


Timer: optimization took 1.180189s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=11 | final_cost=1.604529e-02 | time=1.180s | final_gradient_norm=0.0015533703418106014


Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-06}


Timer: optimization took 0.197119s


...

Convergence: reason=`ftol` termination condition is satisfied. | iterations=38 | final_cost=1.264653e-02 | time=0.251s | final_gradient_norm=241837.47442982648


Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-06}


Timer: optimization took 0.277772s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=33 | final_cost=8.801456e-03 | time=0.278s | final_gradient_norm=12757.225495054028


Temps (C): [10.0, 10.0, 20.0, 30.0, 40.0, 50.0]
{283.15: np.float64(1.114781348122104), 293.15: np.float64(1.0), 303.15: np.float64(2.0716235222245247), 313.15: np.float64(4013.5714885727525), 323.15: np.float64(5966.780997628978)}


## Plot raw vs shifted

In [4]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)
colors = plt.cm.viridis(np.linspace(0, 1, len(datasets)))
for c, data in zip(colors, datasets):
    temp_c = data.metadata['temperature'] - 273.15
    axes[0].loglog(data.x/(2*np.pi), np.real(data.y), 'o', color=c, label=f"{temp_c:.0f}°C G'")
axes[0].set_title("Unshifted G'")
axes[0].set_xlabel('Frequency (Hz)')
axes[0].set_ylabel('Modulus (Pa)')
axes[0].grid(True, which='both', ls='--', alpha=0.4)
axes[0].legend(ncol=2, fontsize=8)

axes[1].loglog(master.x/(2*np.pi), np.real(master.y), 'o', label="G' master", alpha=0.7)
axes[1].loglog(master.x/(2*np.pi), np.imag(master.y), 's', label='G" master', alpha=0.7)
axes[1].set_title('Mastercurve (auto-shift)')
axes[1].grid(True, which='both', ls='--', alpha=0.4)
axes[1].legend()
plt.show()


/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_18854/163117256.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Fit models on mastercurve

In [5]:
omega_master = master.x
G_master = master.y

gm = GeneralizedMaxwell(n_modes=5, modulus_type='tensile')
gm.fit(omega_master, G_master, test_mode='oscillation', use_log_residuals=True)
gm_pred_components = gm.predict(omega_master)
gm_pred = gm_pred_components[:,0] + 1j*gm_pred_components[:,1]
gm_r2 = r2_complex(G_master, gm_pred)

fm_pred = np.full_like(G_master, np.nan)
fm_r2 = np.nan
try:
    fm = FractionalMaxwellModel()
    fm.fit(omega_master, G_master, test_mode='oscillation', use_log_residuals=True)
    fm_pred = fm.predict(omega_master, test_mode='oscillation')
    fm_r2 = r2_complex(G_master, fm_pred)
except Exception as exc:
    print(f"Fractional Maxwell fit failed: {exc}")

print({'gm_r2': gm_r2, 'fm_r2': fm_r2})


Starting least squares optimization | {'method': 'trf', 'n_params': 11, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 0.716901s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=15 | final_cost=1.406711e+06 | time=0.717s | final_gradient_norm=65.75140123766302


Starting least squares optimization | {'method': 'trf', 'n_params': 11, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 0.226168s


...

Element minimization: reducing from 5 to 1 modes


Starting least squares optimization | {'method': 'trf', 'n_params': 4, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 0.928818s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=26 | final_cost=1.430002e+01 | time=0.929s | final_gradient_norm=0.031436418513683693


{'gm_r2': 0.5736802519113267, 'fm_r2': -1.7217816335770597}
